In [ ]:
%matplotlib inline
from matplotlib import pyplot
import geopandas
from shapely.geometry import Polygon
from shapely.affinity import scale
from shapely.affinity import translate
import numpy as np

%run -i weave_map.py

# Exploring possibilities in tiling

In [ ]:
ak = geopandas.read_file("../data/imd-auckland-2018.gpkg")

In [ ]:
import copy


## Wobbly-sided 'Escherian' squares

In [ ]:

d = 200

tile_base = Polygon([(-d, -d), (d, -d), (d, d), (-d, d)])
tile_base = geopandas.GeoDataFrame(
    geometry = geopandas.GeoSeries([tile_base]), crs = ak.crs)

In [ ]:
x1 = y2 = [x for x in range(-d, d+1, 3)]
y1 = [-d + d/4 * np.sin(x / d * np.pi) for x in x1]

x2 = [-_ for _ in y1]
y2 = [_ for _ in x1]

x3 = x1.copy()
x3.reverse()
y3 = y1.copy()
y3.reverse()
y3 = [2*d + _ for _ in y3]

x4 = [-_ for _ in y3]
y4 = [_ for _ in x3]

In [ ]:

side1 = [(x, y) for x, y in zip(x1, y1)]
side2 = [(x, y) for x, y in zip(x2, y2)][1:]
side3 = [(x, y) for x, y in zip(x3, y3)][1:]
side4 = [(x, y) for x, y in zip(x4, y4)][1:]

tile1 = translate(
    scale(Polygon(side1 + side2 + side3 + side4), 0.5, 0.5),-d/2, -d/2)
tile2 = translate(tile1, d, 0)
tile3 = translate(tile2, 0, d)
tile4 = translate(tile3, -d, 0)

tile = geopandas.GeoDataFrame(
    geometry = geopandas.GeoSeries([tile1, tile2, tile3, tile4]),
    data = {"strand": list("abcd")}, crs = ak.crs)

ax = tile.plot(column = "strand", edgecolor = "w")
tile_base.plot(ax = ax, facecolor = "#0000", edgecolor = "k")

In [ ]:
unit = WeaveUnit(crs = 2193) # just to get the container
unit.elements = tile
unit.tile = tile_base

In [ ]:
tiling = Tiling(unit, ak)
t30 = ak.overlay(tiling.rotated(30))
t30.shape

In [ ]:
fig = pyplot.figure(figsize = (24, 15))
ax = fig.add_subplot(111)
# ak.plot(ax = ax, facecolor = "grey", linewidth = 0)

strands = set(t30.strand)
cols = [x for x in t30.columns if "Rank" in x]
cmaps = ("viridis", "plasma_r", "Spectral", "PRGn")

for strand, col, cmap in zip(strands, cols, cmaps):
    t30[t30.strand == strand].plot(ax = ax, column = col, cmap = cmap, k = 100, scheme = "equal_interval", linewidth = 0)


## Concentric annuli

In [ ]:
d = 200
c = [Point((0, 0)).buffer(r) for r in range(d, 0, -d//7)]
annuli = [circ.difference(c[i + 1]) for i, circ in enumerate(c[:-1])] + c[-1:]

tile = geopandas.GeoDataFrame(
    geometry = geopandas.GeoSeries(annuli),
    crs = ak.crs, data = {"strand": list("abcdefg")}
)

### Some tile shapes
Note that only rectangular and hexagonal currently useable in the weave code. Have to add a `_get_tri_centres` method to the `weave_map.TileGrid` class to enable triangles to be tiled.

In [ ]:
def get_regular_polygon(n, d):
    a0 = -90 + 360 / n / 2
    angles = np.linspace(0, 360 * (n - 1) / n, n) + a0
    R = d / np.cos(np.radians(180 / n))
    corners = [(R * np.cos(np.radians(a)), R * np.sin(np.radians(a)))
               for a in angles]
    return Polygon(corners)

tri_base = geopandas.GeoDataFrame(crs = ak.crs,
    geometry = geopandas.GeoSeries([get_regular_polygon(3, d)]))
rect_base = geopandas.GeoDataFrame(crs = ak.crs,
    geometry = geopandas.GeoSeries([get_regular_polygon(4, d)]))
hex_base = geopandas.GeoDataFrame(crs = ak.crs,
    geometry = geopandas.GeoSeries([get_regular_polygon(6, d)]))


In [ ]:
ax = tile.plot(column = "strand")
rect_base.plot(ax = ax, alpha = 0.35)
hex_base.plot(ax = ax, alpha = 0.35)
tri_base.plot(ax = ax, alpha = 0.35)

In [ ]:
unit = WeaveUnit(crs = 2193)
unit.elements = tile
unit.tile = hex_base
unit.tile_shape = "hexagon"

tiling = Tiling(unit, ak)

In [ ]:
t30 = ak.overlay(tiling.rotated(15))

In [ ]:
fig = pyplot.figure(figsize = (24, 15))
ax = fig.add_subplot(111)
ak.plot(ax = ax, facecolor = "#009966", linewidth = 0)

strands = set(t30.strand)
cols = [x for x in t30.columns if "Rank" in x]
cmaps = ("BrBG", "PiYG", "PRGn", "PuOr", "RdGy", "RdYlBu", "Spectral")

for strand, col, cmap in zip(strands, cols, cmaps):
    t30[t30.strand == strand].plot(ax = ax, column = col, cmap = cmap, k = 100, scheme = "equal_interval", linewidth = 0)


In [ ]:
fig.savefig("test.svg")
